In [1]:
import os
import glob

import numpy as np
import pandas as pd
import geopandas as gpd

from sklearn.neighbors import KDTree

In [2]:
obs_files = glob.glob("./OUTPUT/*csv")
idx_lsit = []
concat_df_dict = {}
for obs_file in obs_files:
    file_info = obs_file.split("\\")[-1].split('_')
    road_name, road_dir = file_info[1],file_info[2]

    road_info_file = glob.glob(f'C:/Users/user/Desktop/DB/ROAD/서울&시흥_road_shp/**/*{road_name}_{road_dir}*shp', recursive=True)[0]


    dongbu_d_df = pd.read_csv(obs_file)
    road_info = gpd.read_file(road_info_file, encoding = 'cp949')
    try :
        road_info = road_info[['LANES', 'ROAD_RANK', 'ROAD_TYPE', 'ROAD_USE', 'MULTI_LINK', 'CONNECT', 'MAX_SPD', 'long', 'lat']]
    except:
        road_info = road_info[['LANES', 'ROAD_RANK', 'ROAD_TYPE', 'ROAD_USE', 'MULTI_LINK', 'CONNECT', 'MAX_SPD', 'long_mid','lat_mid']]	
        road_info = road_info.rename({'long_mid':'long','lat_mid':'lat'}, axis = 1)
    road_info = road_info.dropna()

    observ_line = np.array(list(zip(road_info['long'], road_info['lat'])))
    model_line = np.array(list(zip(dongbu_d_df['longitude'], dongbu_d_df['latitude'])))

    tree = KDTree(observ_line)
    # 각 점마다 가장 가까운 점을 찾아 매칭

    matched_points = []
    for point in model_line:
        _, index = tree.query([point], k=1)  # k=1로 설정하여 가장 가까운 점 하나만 선택
        matched_points.append(observ_line[index[0]])

    matched_lon = [point[0][0] for point in matched_points]
    matched_lat = [point[0][1] for point in matched_points]

    df = pd.DataFrame({'longitude': model_line[:, 0], 'latitude': model_line[:, 1],
                        'long_match': matched_lon, 'lat_match': matched_lat})

    unique_coords = df[['longitude', 'latitude','long_match','lat_match']].drop_duplicates()

    total_df = pd.merge(unique_coords,dongbu_d_df, on=['longitude','latitude'], how='right')
    road_info = road_info.rename({'long':'long_match','lat':'lat_match'}, axis = 1)
    total_df = pd.merge(total_df, road_info, on=['long_match', 'lat_match'], how='left')
    print(obs_file)
    concat_df_dict[f"{obs_file}"] = total_df
    idx_lsit.append(f"{obs_file}")
    del road_info_file, total_df, road_info


./OUTPUT\seoul_dongbu_D_202301_26_lufft_concat.csv
./OUTPUT\seoul_dongbu_D_202301_27_lufft_concat.csv
./OUTPUT\seoul_dongbu_U_202301_19_vaisala_concat.csv
./OUTPUT\seoul_gangbyeon_U_202301_19_vaisala_concat.csv
./OUTPUT\seoul_naebu_U_202301_19_vaisala_concat.csv
./OUTPUT\seoul_olympic_D_202301_26_lufft_concat.csv
./OUTPUT\seoul_olympic_D_202301_27_lufft_concat.csv
./OUTPUT\seoul_olympic_U_202301_17_lufft_concat.csv
./OUTPUT\seoul_olympic_U_202301_18_lufft_concat.csv
./OUTPUT\seoul_olympic_U_202301_19_lufft_concat.csv
./OUTPUT\seoul_seobu_D_202301_20_lufft_concat.csv
./OUTPUT\sihueng_SHR1_D_202301_20_lufft_concat.csv
./OUTPUT\sihueng_SHR2_U_202301_20_lufft_concat.csv
./OUTPUT\sihueng_SHR3_D_202301_20_lufft_concat.csv
./OUTPUT\sihueng_SHR4_U_202301_20_lufft_concat.csv


In [3]:
concat_df_dict[idx_lsit[1]].columns

Index(['longitude', 'latitude', 'long_match', 'lat_match', 'lon', 'lat',
       'timestamp', 'surface_temperature', 'day', 'hour', 'date_time',
       'update_time', 'p_hour', 'road_name', 'direction', 'jr', 'mg', 'org',
       'ss', 'yc', 'closest_value', 'closest_column', 'LANES', 'ROAD_RANK',
       'ROAD_TYPE', 'ROAD_USE', 'MULTI_LINK', 'CONNECT', 'MAX_SPD'],
      dtype='object')

In [4]:

from sklearn import metrics

In [5]:
for i in idx_lsit:
    df = concat_df_dict[i]

    for road in df['road_name'].unique():
        for hour in df['LANES'].unique():
            try:
                df_hour = df[(df['road_name']==road) & (df['LANES']==hour)]
                print(road , hour)
                print(metrics.mean_squared_error(df_hour['surface_temperature'],df_hour['org']), end = "|")
                print(metrics.mean_squared_error(df_hour['surface_temperature'],df_hour['yc']), end = "|")
                print(metrics.mean_squared_error(df_hour['surface_temperature'],df_hour['mg']), end = "|")
                print(metrics.mean_squared_error(df_hour['surface_temperature'],df_hour['jr']), end = "|")
                print(metrics.mean_squared_error(df_hour['surface_temperature'],df_hour['ss']), end = "|")
                print()
                # print(df_hour.value_counts('closest_column'))
            except:
                print()

dongbu 3
14.45834237731093|0.7942782668620778|19.176037796887833|20.85242992397612|0.9138040933134924|
dongbu 3
6.297939255344055|25.353831756336263|3.937838986953362|4.130976588627521|26.0082453477543|
dongbu 2
2.747930972858988|5.698337559712366|21.374168917290444|6.25628585362434|8.19905175495932|
dongbu 4
3.4979810108445406|12.6150619911161|11.348660180770553|4.150830077046804|16.148452839251934|
dongbu 3
2.8442242817984957|18.461825615831795|7.561315507358431|2.765369470197977|22.948975259469737|
dongbu 1
3.8396952659715042|11.00152666826825|12.874170473269317|3.8205829821622053|14.223793912227972|
gangbyeon 3
2.5945040273671824|4.100556813003441|66.29832902049091|28.89021236951772|1.777558768762876|
gangbyeon 4
2.051799651266136|1.7631539675881973|50.95822306513178|19.219241929502576|0.979875685474445|
gangbyeon 5
1.3051717239078404|1.0672718527718215|49.355243636221914|18.051203632329127|0.38660704318133887|
gangbyeon 2
3.1385146789347083|2.00321051276523|56.28223244546606|22.18

In [56]:
file_info = obs_file[0].split("\\")[-1].split('_')
road_name, road_dir = file_info[1],file_info[2]

road_info_file = glob.glob(f'C:/Users/user/Desktop/DB/ROAD/서울&시흥_road_shp/seoul/*{road_name}_{road_dir}*shp')[0]

In [57]:
dongbu_d_df = pd.read_csv(glob.glob("./OUTPUT/*csv")[0])

In [58]:
road_info = gpd.read_file(road_info_file, encoding = 'cp949')

In [59]:
road_info_1 = road_info[['LANES', 'ROAD_RANK', 'ROAD_TYPE', 'ROAD_USE', 'MULTI_LINK', 'CONNECT', 'MAX_SPD', 'long', 'lat']]
road_info_1 = road_info_1.dropna()

In [30]:
observ_line = np.array(list(zip(road_info_1['long'], road_info_1['lat'])))
model_line = np.array(list(zip(dongbu_d_df['longitude'], dongbu_d_df['latitude'])))

tree = KDTree(observ_line)
# 각 점마다 가장 가까운 점을 찾아 매칭

matched_points = []
for point in model_line:
    _, index = tree.query([point], k=1)  # k=1로 설정하여 가장 가까운 점 하나만 선택
    matched_points.append(observ_line[index[0]])

matched_lon = [point[0][0] for point in matched_points]
matched_lat = [point[0][1] for point in matched_points]

df = pd.DataFrame({'longitude': model_line[:, 0], 'latitude': model_line[:, 1],
                       'long_match': matched_lon, 'lat_match': matched_lat})

unique_coords = df[['longitude', 'latitude','long_match','lat_match']].drop_duplicates()

total_df = pd.merge(unique_coords,dongbu_d_df, on=['longitude','latitude'], how='right')
road_info_1 = road_info_1.rename({'long':'long_match','lat':'lat_match'}, axis = 1)
total_df = pd.merge(total_df, road_info_1, on=['long_match', 'lat_match'], how='left')

In [31]:
total_df

,longitude,latitude,long_match,lat_match,lon,lat,timestamp,surface_temperature,day,hour,...,yc,closest_value,closest_column,LANES,ROAD_RANK,ROAD_TYPE,ROAD_USE,MULTI_LINK,CONNECT,MAX_SPD
0,127.066533,37.516700,127.065366,37.519478,127.066554,37.516691,2023-01-26 11:10:00,-2.5121,26,11,...,-3.423115,-3.423115,yc,3,102,000,0,0,000,80
1,127.066533,37.516700,127.065366,37.519478,127.066554,37.516691,2023-01-26 11:10:00,-2.5272,26,11,...,-3.423115,-3.423115,yc,3,102,000,0,0,000,80
2,127.066533,37.516700,127.065366,37.519478,127.066554,37.516691,2023-01-26 11:10:00,-2.5728,26,11,...,-3.423115,-3.423115,yc,3,102,000,0,0,000,80
3,127.066533,37.516700,127.065366,37.519478,127.066554,37.516691,2023-01-26 11:10:00,-2.6184,26,11,...,-3.423115,-3.423115,yc,3,102,000,0,0,000,80
4,127.066533,37.516700,127.065366,37.519478,127.066554,37.516691,2023-01-26 11:10:00,-2.6097,26,11,...,-3.423115,-3.423115,yc,3,102,000,0,0,000,80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2523,127.122250,37.472083,127.106450,37.488293,127.122036,37.472161,2023-01-26 11:17:00,-2.5634,26,11,...,-3.330996,-3.330996,yc,3,102,000,0,0,000,80
2524,127.122250,37.472083,127.106450,37.488293,127.122036,37.472161,2023-01-26 11:17:00,-2.5916,26,11,...,-3.330996,-3.330996,yc,3,102,000,0,0,000,80
2525,127.122250,37.472083,127.106450,37.488293,127.122036,37.472161,2023-01-26 11:17:00,-2.5808,26,11,...,-3.330996,-3.330996,yc,3,102,000,0,0,000,80
2526,127.122250,37.472083,127.106450,37.488293,127.122036,37.472161,2023-01-26 11:17:00,-2.5982,26,11,...,-3.330996,-3.330996,yc,3,102,000,0,0,000,80
